<a href="https://colab.research.google.com/github/Abhishek3102/ML/blob/main/Dask_Text_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "dask[complete]"

In [2]:
from dask.distributed import Client, progress

client = Client(n_workers=2, threads_per_worker=2, memory_limit='2GB')
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/scheduler-40pejg0o', purging
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/worker-zlv99pi6', purging
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43929
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40127'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33413'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:44801 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44801
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:3

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 3.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43929,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 3.73 GiB
Comm: tcp://127.0.0.1:40313,Total threads: 2
Dashboard: http://127.0.0.1:38817/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:40127,


In [3]:
import sklearn.datasets

bunch = sklearn.datasets.fetch_20newsgroups()

In [4]:
import dask.dataframe as dd
import pandas as pd

df = dd.from_pandas(pd.DataFrame({"text": bunch.data, "target": bunch.target}),
                    npartitions=25)
print(df)

Dask DataFrame Structure:
                  text target
npartitions=25               
0               string  int64
453                ...    ...
...                ...    ...
10862              ...    ...
11313              ...    ...
Dask Name: frompandas, 1 expression
Expr=df


In [5]:
print(df.head().loc[0, 'text'][:500])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a m


In [6]:
!pip install dask_ml

In [8]:
import dask_ml.feature_extraction.text
import dask.array as da

vect = dask_ml.feature_extraction.text.HashingVectorizer()
# Convert the dask Series to a dask Array and fill missing values with an empty string
X = vect.fit_transform(df['text'].fillna('').to_dask_array(lengths=True))
print(X)

/usr/local/lib/python3.11/dist-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 21.14 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/dask/dataframe/dask_expr/_collection.py:1428: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(


dask.array<_transformer, shape=(nan, 1048576), dtype=float64, chunksize=(nan, 1048576), chunktype=scipy.csr_matrix>


In [9]:
X.blocks[0].compute()

<453x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 64357 stored elements in Compressed Sparse Row format>

In [10]:
bunch.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [11]:
import numpy as np

positive = np.arange(len(bunch.target_names))[['comp' in x for x in bunch.target_names]]
y = df['target'].isin(positive).astype(int)
print(y)

Dask Series Structure:
npartitions=25
0        int64
453        ...
         ...  
10862      ...
11313      ...
Dask Name: astype, 5 expressions
Expr=AsType(frame=Isin(frame=df['target'], values=_DelayedExpr(Delayed('delayed-e7e2ec8873f0ffb659a212f8205e6f54'))), dtypes=<class 'int'>)


In [12]:
import numpy as np
import sklearn.linear_model
import sklearn.pipeline

import dask_ml.wrappers

sgd = sklearn.linear_model.SGDClassifier(tol=1e-3)
clf = dask_ml.wrappers.Incremental(sgd, scoring='accuracy', assume_equal_chunks=True)
pipe = sklearn.pipeline.make_pipeline(vect, clf)

In [ ]:
import sklearn.impute
imputer = sklearn.impute.SimpleImputer(strategy='constant', fill_value='')
pipe = sklearn.pipeline.make_pipeline(imputer, vect, clf)
pipe.fit(df['text'].to_dask_array(lengths=True), y, incremental__classes=[0, 1])

In [ ]:
predictions = pipe.predict(df['text'])
predictions

In [ ]:
dask_ml.metrics.accuracy_score(y, predictions)